# Select batch-id for execution

In [ ]:
batch_id = "e99f652f-6258-49e4-b963-109ed5d63a8f"

#### imports

In [ ]:
import asyncio
import json
import math
import sys
from datetime import date, datetime, timedelta

import alpaca_trade_api as tradeapi
import iso8601
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import pandas as pd
import pytz
import requests
from dateutil import parser
from IPython.display import HTML, display, Markdown
from liualgotrader.analytics.analysis import (
    calc_batch_revenue,
    count_trades,
    load_trades_by_batch_id,
    trades_analysis,
    symobl_trade_analytics,
)

from liualgotrader.common import config
from liualgotrader.common.market_data import (
    get_historical_data_from_poylgon_for_symbols,
)
from pandas import DataFrame as df
from pytz import timezone
from talib import BBANDS, MACD, RSI
from liualgotrader.common.data_loader import DataLoader 
%matplotlib inline

nest_asyncio.apply()

#### connect to Alpaca Markets

In [ ]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")

#### Load batch data

In [ ]:
trades = load_trades_by_batch_id(batch_id)

In [ ]:
trades

In [ ]:
if trades.empty:
    assert False, "Empty batch. halting execution."

## Calculating batch performance

In [ ]:
how_was_my_batch = trades_analysis(trades, batch_id)

In [ ]:
pd.set_option("display.max_rows", how_was_my_batch.shape[0] + 1)
revenues = round(sum(how_was_my_batch["revenues"]), 2)
traded = len(how_was_my_batch)
win_ratio = round(
        1.0
        * len(how_was_my_batch[how_was_my_batch.revenues >= 0])
        / len(how_was_my_batch[how_was_my_batch.revenues < 0]),
        2,
    ) if len(how_was_my_batch[how_was_my_batch.revenues < 0]) else "high!"
print(
    f"Total revenues=${revenues}\nTotal traded:{traded} Win/Lose ration {win_ratio}"
)

display(
    Markdown(f"{len(how_was_my_batch[how_was_my_batch.revenues >= 0])} **Winners**")
)
display(how_was_my_batch[how_was_my_batch.revenues >= 0].sort_values('revenues'))
display(Markdown(f"{len(how_was_my_batch[how_was_my_batch.revenues < 0])} **Lossers**"))
display(how_was_my_batch[how_was_my_batch.revenues < 0].sort_values('revenues'))

## Display trades in details

In [ ]:
minute_history = DataLoader()
for symbol in trades.symbol.unique():
    symbol_df = trades.loc[trades["symbol"] == symbol]
    start_date = symbol_df["client_time"].min()
    start_date = start_date.replace(hour=9, minute=30)
    end_date =  symbol_df["client_time"].max().replace(hour=16, minute=0)
    cool_down_date = start_date + timedelta(minutes=5)
    symbol_data = minute_history[symbol][start_date:end_date]
    minute_history_index = symbol_data.close.index.get_loc(
        start_date, method="nearest"
    )
    end_index = symbol_data.close.index.get_loc(
        end_date, method="nearest"
    )
    cool_minute_history_index = symbol_data["close"].index.get_loc(
        cool_down_date, method="nearest"
    )
    open_price = symbol_data.close[cool_minute_history_index]
    plt.plot(
        symbol_data.close[minute_history_index:end_index].between_time(
            "9:30", "16:00"
        ),
        label=symbol,
    )
    plt.xticks(rotation=45)

    d, profit = symobl_trade_analytics(symbol_df, open_price, plt)
    
    print(f"{symbol} analysis with profit {round(profit, 2)}")
    display(HTML(pd.DataFrame(data=d).to_html()))
    plt.legend()
    plt.show()